<a href="https://www.kaggle.com/code/aleksandrmorozov123/computer-vision-with-pytorch?scriptVersionId=153609098" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cifar10-python/cifar-10-python.tar.gz
/kaggle/input/cifar10-python/cifar-10-batches-py/data_batch_1
/kaggle/input/cifar10-python/cifar-10-batches-py/data_batch_2
/kaggle/input/cifar10-python/cifar-10-batches-py/batches.meta
/kaggle/input/cifar10-python/cifar-10-batches-py/test_batch
/kaggle/input/cifar10-python/cifar-10-batches-py/data_batch_3
/kaggle/input/cifar10-python/cifar-10-batches-py/data_batch_5
/kaggle/input/cifar10-python/cifar-10-batches-py/data_batch_4
/kaggle/input/cifar10-python/cifar-10-batches-py/readme.html


**Begin with foundations - tensors**

In [2]:
import torch
x = torch.tensor ([[1,2,3], [4,5,6]])
y = torch.tensor ([[7,8,9], [10,11,12]])
z = x + y
print (z)

tensor([[ 8, 10, 12],
        [14, 16, 18]])


**Create the tensors**

In [3]:
import numpy 

# created from preexisting arrays
w = torch.tensor ([1,2,3])                 # from a list
w = torch.tensor ([1,2,3])                 # from a tuple
w = torch.tensor (numpy.array ([1,2,3]))   # from a numpy.array

# initialized by size
w = torch.empty (100, 200)                 # uninitialized, element values are not predictable
w = torch.zeros (100, 200)                 # all elements initialized with 0.0
w = torch.ones (100, 200)                  # all elements initialized with 1.0

# initialized by size with random values
w = torch.rand (100, 200)
w = torch.randn (100, 200)
w = torch.randint (5, 10, (100, 200))

# initialized to have the same size, data type and device as another tensor
x = torch.empty_like (w)

# specify the data type at creation using dtype
w = torch.tensor ([1,2,3], dtype = torch.float32)

# use the casting method to cast to a new data type
w.int ()        # w remains a float 32 after the cast
w = w.int ()    # w changes to an int32 after the cast

# use the to() method to cast to a new type
w = w.to (torch.float64)
w = w.to (dtype = torch.float64)

# Python automatucally converts data types during operations
x = torch.tensor ([1,2,3], dtype = torch.int32)
y = torch.tensor ([1,2,3], dtype = torch.float32)
z = x + y
print (z.dtype)

torch.float32


**Indexing, sdlicing, combining and splitting tensors**

In [4]:
x = torch.tensor ([[1, 2], [3, 4], [5, 6], [7, 8]])
print (x)

tensor([[1, 2],
        [3, 4],
        [5, 6],
        [7, 8]])


In [5]:
# indexing, returns a tensor
print (x[1, 1])

tensor(4)


In [6]:
# indexing, returns a value as a Python number
print (x[1,1].item ())

4


In [7]:
# slicing
print (x[:2, 1])

tensor([2, 4])


In [8]:
# Boolean indexing
# only keeps elements less than 4
print (x [x < 4])

tensor([1, 2, 3])


In [9]:
# transpose array
print (x.t())

tensor([[1, 3, 5, 7],
        [2, 4, 6, 8]])


In [10]:
# change shape
print (x.view ((2,4)))

tensor([[1, 2, 3, 4],
        [5, 6, 7, 8]])


In [11]:
# combining tensors 
y = torch.stack ((x, x))
print (y)

tensor([[[1, 2],
         [3, 4],
         [5, 6],
         [7, 8]],

        [[1, 2],
         [3, 4],
         [5, 6],
         [7, 8]]])


In [12]:
# splitting tensors 
a, b = x.unbind (dim = 1)
print (a, b)

tensor([1, 3, 5, 7]) tensor([2, 4, 6, 8])


**Automatic differentiation**

In [13]:
x = torch.tensor ([[1,2,3], [4,5,6]], 
                 dtype = torch.float, requires_grad = True)
print (x)

tensor([[1., 2., 3.],
        [4., 5., 6.]], requires_grad=True)


In [14]:
f = x.pow(2).sum()
print (f)

tensor(91., grad_fn=<SumBackward0>)


In [15]:
f.backward ()
print (x.grad)  # df/dx = 2x

tensor([[ 2.,  4.,  6.],
        [ 8., 10., 12.]])
